In [1]:
# A few standard library imports
import itertools, math, collections
# nice printing
from IPython.display import display, clear_output

In [2]:
def downloadData(day):  
    import browsercookie
    import urllib.request
    cj = browsercookie.firefox()
    opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
    assert b'watercrossing' in opener.open("https://adventofcode.com/2022/").read()
    data = opener.open("https://adventofcode.com/2022/day/%d/input" %day).read()
    with open("data-2022/day%dinput.txt" %day, 'wb') as f:
        f.write(data)

# Day 1

In [3]:
with open("data-2022/day1input.txt", 'r') as f:
    day1data = f.read()
day1data = [[int(y) for y in x.split()] for x in day1data.split("\n\n")]

In [4]:
max(sum(x) for x in day1data)

64929

In [5]:
sum(sorted(sum(x) for x in day1data)[-3:])

193697

# Day 2